In [1]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm 
from datasets import load_dataset

import seaborn as sns
sns.set(style='darkgrid')
sns.set_palette(sns.color_palette('husl', 8))

import matplotlib.pyplot as plt
%matplotlib inline

params = {
    'legend.fontsize': 'x-large',
    'figure.figsize': (16, 5),
    'axes.labelsize': 'x-large',
    'axes.titlesize':'x-large',
    'xtick.labelsize':'x-large',
    'ytick.labelsize':'x-large',
}
plt.rcParams.update(params)

from src.loading import huggingface_dataset_to_dataframes
from src.bias import Factors

from IPython.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [2]:
# Human learning Datasets
dataset_comparisons = load_dataset(
    path="openai/summarize_from_feedback",
    name='comparisons',
)

dataset_axis = load_dataset(
    path="openai/summarize_from_feedback",
    name='axis',
)

Reusing dataset summarize_from_feedback (C:\Users\Usuario\.cache\huggingface\datasets\openai___summarize_from_feedback\comparisons\0.0.0\483f970ceb55b926b0a087ef4f678ab1b089bc8174a107a452c6152e88af7ff0)


  0%|          | 0/2 [00:00<?, ?it/s]

Reusing dataset summarize_from_feedback (C:\Users\Usuario\.cache\huggingface\datasets\openai___summarize_from_feedback\axis\0.0.0\483f970ceb55b926b0a087ef4f678ab1b089bc8174a107a452c6152e88af7ff0)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# Axis dataset
test_axis, validation_axis = huggingface_dataset_to_dataframes(
    dataset_axis, list(dataset_axis.keys()), dict_cols=["info", "summary", "axes"]
)

# Comparisions dataset
train_comparisons, validation_comparisons = huggingface_dataset_to_dataframes(
    dataset_comparisons,
    list(dataset_comparisons.keys()),
    dict_cols=["info", "extra"],
    list_col="summaries",
)

# Re-order data
axis_columns = [
    "worker",
    "batch",
    "split",
    "id",
    "source",
    "subsource",
    "title",
    "text",
    "summary",
    "policy",
    "note",
    "compatible",
    "accuracy",
    "coverage",
    "coherence",
    "overall",
]

comparison_columns = [
    "worker",
    "batch",
    "split",
    "id",
    "source",
    "subsource",
    "title",
    "text",
    "summary_0",
    "policy_0",
    "note_0",
    "summary_1",
    "policy_1",
    "note_1",
    "confidence",
    "choice",
]

# Save datasets
test_axis[axis_columns].to_pickle("data/raw/test_axis.pkl")
validation_axis[axis_columns].to_pickle("data/raw/validation_axis.pkl")

train_comparisons[comparison_columns].to_pickle("data/raw/train_comparisons.pkl")
validation_comparisons[comparison_columns].to_pickle("data/raw/validation_comparisons.pkl")

In [4]:
test_axis

,worker,batch,split,id,title,summary,policy,note,overall,accuracy,coverage,coherence,compatible,source,subsource,text
0,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,cnndm1,test,167f80cc6634b166a699d182e25c81a2349d82d2,Newcastle United midfielder Moussa Sissoko fac...,Moussa Sissoko was sent off against Liverpool ...,ref,"Misleading: ""Carver admits he is only concerne...",3,5,4,2.0,None,cnn_dailymail,dailymail,Newcastle stand-in skipper Moussa Sissoko is f...
1,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,cnndm1,test,167f80cc6634b166a699d182e25c81a2349d82d2,Newcastle United midfielder Moussa Sissoko fac...,Newcastle have a mountain to climb in the Prem...,sup4_t0.7,"""Carver is determined to make the most of his ...",1,3,1,1.0,None,cnn_dailymail,dailymail,Newcastle stand-in skipper Moussa Sissoko is f...
2,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,cnndm1,test,167f80cc6634b166a699d182e25c81a2349d82d2,Newcastle United midfielder Moussa Sissoko fac...,Moussa Sissoko has been sent off following a d...,sup4_ppo_rm4_t.7,"""Carver feels the player should have been sent...",2,4,2,2.0,None,cnn_dailymail,dailymail,Newcastle stand-in skipper Moussa Sissoko is f...
3,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,cnndm1,test,167f80cc6634b166a699d182e25c81a2349d82d2,Newcastle United midfielder Moussa Sissoko fac...,Moussa Sissoko is facing a disciplinary action...,pretrain_xl_t.7,Doesnt summarize important points. Match info ...,3,7,3,6.0,None,cnn_dailymail,dailymail,Newcastle stand-in skipper Moussa Sissoko is f...
4,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,cnndm1,test,167f80cc6634b166a699d182e25c81a2349d82d2,Newcastle United midfielder Moussa Sissoko fac...,Newcastle need to start helping themselves now...,sup4_6b_t0.7,Completely misses main info.,1,7,1,3.0,None,cnn_dailymail,dailymail,Newcastle stand-in skipper Moussa Sissoko is f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6307,uvzut5OK2bvei9zoCDdktcfLENYioY,cnndm4,test,843f85685d2ca5bab950e30d6cd89e91bc539018,Lamb born twice the normal size weighing 20lbs...,This bundle of joy was a special spring surpri...,lead3,•great summary with elaborate detail.,7,7,7,7.0,True,cnn_dailymail,dailymail,This bundle of joy was a special spring surpri...
6308,uvzut5OK2bvei9zoCDdktcfLENYioY,cnndm4,test,843f85685d2ca5bab950e30d6cd89e91bc539018,Lamb born twice the normal size weighing 20lbs...,Big Ben is twice the weight of the average new...,supcnndm1_6b,•[He is the biggest lamb ever born in the UK.]...,4,4,5,7.0,True,cnn_dailymail,dailymail,This bundle of joy was a special spring surpri...
6309,uvzut5OK2bvei9zoCDdktcfLENYioY,cnndm4,test,843f85685d2ca5bab950e30d6cd89e91bc539018,Lamb born twice the normal size weighing 20lbs...,Shepherd John Hendy and team of three helpers ...,supcnndm3_6b,•sufficient key information but minimal details.,4,7,5,7.0,True,cnn_dailymail,dailymail,This bundle of joy was a special spring surpri...
6310,uvzut5OK2bvei9zoCDdktcfLENYioY,cnndm4,test,843f85685d2ca5bab950e30d6cd89e91bc539018,Lamb born twice the normal size weighing 20lbs...,This bundle of joy was a special spring surpri...,pretrain_xl,•good summary with average detail.,6,7,6,7.0,True,cnn_dailymail,dailymail,This bundle of joy was a special spring surpri...


In [5]:
validation_axis

,worker,batch,split,id,title,summary,policy,note,overall,accuracy,coverage,coherence,compatible,source,subsource,text
0,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,tldraxis1,valid2,t3_4l0bal,19f with fiance 20m and roommate 19m- fiance a...,Fiance and I recently got infected with scabi...,sup4_ppo_rm4_t.7,'our apartment will not go away. I'm afraid he...,5.0,5.0,6.0,5.0,None,reddit,relationship_advice,"Recently, my fiance (20 m) and I (19f) moved ..."
1,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,tldraxis1,valid2,t3_4l0bal,19f with fiance 20m and roommate 19m- fiance a...,my fiance and I refuse to treat our room mate...,pretrain_6b_t.7,None,4.0,4.0,4.0,7.0,None,reddit,relationship_advice,"Recently, my fiance (20 m) and I (19f) moved ..."
2,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,tldraxis1,valid2,t3_4l0bal,19f with fiance 20m and roommate 19m- fiance a...,"fiancé and I contracted scabies, roommate ref...",sup4_6b_ppo_rm4_6b_t.7,the question s missing but the summary is good,6.0,6.0,6.0,7.0,None,reddit,relationship_advice,"Recently, my fiance (20 m) and I (19f) moved ..."
3,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,tldraxis1,valid2,t3_4l0bal,19f with fiance 20m and roommate 19m- fiance a...,"fiance and I are infected with scabies, room ...",sup4_6b_t0.7,"a small inaccuracy and omission, otherwise good",6.0,6.0,6.0,7.0,None,reddit,relationship_advice,"Recently, my fiance (20 m) and I (19f) moved ..."
4,iL7GfrbN2PeB3KInidqSxUdxYcTZmG,tldraxis1,valid2,t3_4l0bal,19f with fiance 20m and roommate 19m- fiance a...,"Fiance and I contracted scabies, roommate ref...",sup4_12b_t0.7,"a small inaccuracy, otherwise good",6.0,5.0,7.0,7.0,None,reddit,relationship_advice,"Recently, my fiance (20 m) and I (19f) moved ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8580,uvzut5OK2bvei9zoCDdktcfLENYioY,tldraxis2,valid2,t3_3i230d,[Update 2] I [18 M] want to ask out a girl [18...,"I want to ask out a girl on a date, general t...",sup4_12b,•summary has added info.,6.0,5.0,7.0,7.0,True,reddit,relationships,"[Original](\n(Clarification on this one, I did..."
8581,uvzut5OK2bvei9zoCDdktcfLENYioY,tldraxis2,valid2,t3_3i230d,[Update 2] I [18 M] want to ask out a girl [18...,"Girl ignored me again, I cease conversation. ...",ref,•summary is completely made up.,1.0,1.0,1.0,7.0,True,reddit,relationships,"[Original](\n(Clarification on this one, I did..."
8582,uvzut5OK2bvei9zoCDdktcfLENYioY,tldraxis2,valid2,t3_3i230d,[Update 2] I [18 M] want to ask out a girl [18...,[Update 2] I [18 M] want to ask out a girl [1...,title,•complete summary.,7.0,7.0,7.0,7.0,True,reddit,relationships,"[Original](\n(Clarification on this one, I did..."
8583,uvzut5OK2bvei9zoCDdktcfLENYioY,tldraxis2,valid2,t3_3i230d,[Update 2] I [18 M] want to ask out a girl [18...,"[Original](\n(Clarification on this one, I di...",lead2,•summary is just an introduction from the orig...,1.0,7.0,1.0,7.0,True,reddit,relationships,"[Original](\n(Clarification on this one, I did..."


In [6]:
train_comparisons

,choice,worker,batch,split,id,title,confidence,summary_0,note_0,policy_0,summary_1,note_1,policy_1,source,subsource,text
0,1,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,t3_34xale,Mother [51] not speaking to me [21] because of...,NaN,Mum is mad at me for not flying on my own tri...,None,sup1,I have made sure my mother is comfortable wit...,None,sup1,reddit,relationships,My boyfriend and I are long distance. We have ...
1,1,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,t3_34xale,Mother [51] not speaking to me [21] because of...,NaN,I have made sure my mother is comfortable wit...,None,sup1,mum isn't speaking to me because I booked a f...,None,ref,reddit,relationships,My boyfriend and I are long distance. We have ...
2,0,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,t3_34xale,Mother [51] not speaking to me [21] because of...,NaN,mum isn't speaking to me because I booked a f...,None,ref,Mum thought I was going to road trip with my ...,None,sup1,reddit,relationships,My boyfriend and I are long distance. We have ...
3,0,qo6WIyEh27cwAjWpA3Q60J7NaDxzQJ,batch3,train,t3_34xale,Mother [51] not speaking to me [21] because of...,NaN,Mum thought I was going to road trip with my ...,None,sup1,Mum is mad at me for not flying on my own tri...,None,sup1,reddit,relationships,My boyfriend and I are long distance. We have ...
4,1,LjvoXOAj5op3WqNnn5b7TZTG8mK7gM,batch3,train,t3_1zwek5,Can I sue my property management company and l...,NaN,My landlord is harassing me and my neighbours...,None,sup1,landlord pretended to be another tenant and w...,None,ref,reddit,AskReddit,My landlord left a falsified message taped to ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92820,1,44Z8ttpKcY6Kr1sNymNnBA0nL0h4dZ,batch9,train,t3_2n5vfw,TIFU by trying to get out of an assignment,9.0,Thought about trying to get out of work by br...,"ok, but strange concern: getting out of work a...",ref,"tried to get out of an assignment, ended up c...",ok,sup4_bo64_rm3,reddit,tifu,"So.. TIFU about fifteen minutes ago, when I wa..."
92821,0,44Z8ttpKcY6Kr1sNymNnBA0nL0h4dZ,batch9,train,t3_2n5vfw,TIFU by trying to get out of an assignment,3.0,TIFU by accidentily spilling half a glass of ...,strange,sup4_ppo_rm3_kl10,TIFU by trying to get out of an assignment by...,"""... my work is on Dropbox on all my spare tim...",sup4_ppo_rm3_kl10,reddit,tifu,"So.. TIFU about fifteen minutes ago, when I wa..."
92822,1,44Z8ttpKcY6Kr1sNymNnBA0nL0h4dZ,batch9,train,t3_2n5vfw,TIFU by trying to get out of an assignment,9.0,TIFU by trying to get out of an assignment by...,"""... my work is on Dropbox on all my spare tim...",sup4_ppo_rm3_kl10,"tried to get out of an assignment, ended up c...",ok,sup4_bo64_rm3,reddit,tifu,"So.. TIFU about fifteen minutes ago, when I wa..."
92823,1,44Z8ttpKcY6Kr1sNymNnBA0nL0h4dZ,batch9,train,t3_2n5vfw,TIFU by trying to get out of an assignment,7.0,Thought about trying to get out of work by br...,"ok, but strange concern: getting out of work a...",ref,TIFU by accidentily spilling half a glass of ...,strange,sup4_ppo_rm3_kl10,reddit,tifu,"So.. TIFU about fifteen minutes ago, when I wa..."


In [7]:
validation_comparisons

,choice,worker,batch,split,id,title,confidence,summary_0,note_0,policy_0,summary_1,note_1,policy_1,source,subsource,text
0,1,p4Oh7rUGyLe1EpilJFWr9sPDpkO016,batch10,valid1,t3_52mb8y,I [23F] have just come out of 8 year relations...,6,"Just came out of 8 year relationship, don't k...",Ok,sup4_t0.7,"I never dated/flirted as an adult, now I'm no...",Ok,sup4_t0.7,reddit,relationships,This is my first post so please be kind :)\n\n...
1,1,p4Oh7rUGyLe1EpilJFWr9sPDpkO016,batch10,valid1,t3_52mb8y,I [23F] have just come out of 8 year relations...,7,Don't know how to date/flirt/flirt with someo...,Repetitive,sup4_t0.7,"Just came out of 8 year relationship, don't k...",Ok,sup4_t0.7,reddit,relationships,This is my first post so please be kind :)\n\n...
2,1,p4Oh7rUGyLe1EpilJFWr9sPDpkO016,batch10,valid1,t3_52mb8y,I [23F] have just come out of 8 year relations...,7,Don't know how to date/flirt/flirt with someo...,Repetitive,sup4_t0.7,"I never dated/flirted as an adult, now I'm no...",Ok,sup4_t0.7,reddit,relationships,This is my first post so please be kind :)\n\n...
3,0,p4Oh7rUGyLe1EpilJFWr9sPDpkO016,batch10,valid1,t3_52mb8y,I [23F] have just come out of 8 year relations...,4,"Just came out of 8 year relationship, don't k...",Ok,sup4_t0.7,Started a long-term relationship as a teenage...,Ok,ref,reddit,relationships,This is my first post so please be kind :)\n\n...
4,1,p4Oh7rUGyLe1EpilJFWr9sPDpkO016,batch10,valid1,t3_52mb8y,I [23F] have just come out of 8 year relations...,6,Don't know how to date/flirt/flirt with someo...,Repetitive,sup4_t0.7,Started a long-term relationship as a teenage...,Ok,ref,reddit,relationships,This is my first post so please be kind :)\n\n...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86081,0,soP7Okyp0e1WdYxQC9IldpwnznT6V1,edit_b2_eval_test,valid2,None,Me [27 F] with my boyfriend [24 M] for four mo...,2,"Boyfriend talks very slowly, I feel frustrated...",None,human_editor,"Boyfriend talks very slowly, I feel frustrated...",None,sup4_6b_t0.7,reddit,r/relationships,I don't know how to ask this without it soundi...
86082,0,soP7Okyp0e1WdYxQC9IldpwnznT6V1,edit_b2_eval_test,valid1,None,Almost every career in the medical field is sa...,2,Medical field looks like its going to hell. Wh...,None,human_editor,Life sucks if you don't have money. Medical fi...,None,ref,reddit,r/Advice,I honestly do not care anymore. \n\nExcluding ...
86083,0,soP7Okyp0e1WdYxQC9IldpwnznT6V1,edit_b2_eval_test,valid2,None,I'm [26/f] in US and my best friend [32/m] is ...,2,"Met a guy on reddit, became best friends, met,...",None,human_editor,"Met a guy on reddit, became best friends, met,...",None,ref,reddit,r/relationship_advice,Someone I consider to be a best friend and I a...
86084,1,soP7Okyp0e1WdYxQC9IldpwnznT6V1,edit_b2_eval_test,valid2,None,Should I [17F] tell him [18M] about an extreme...,8,Had a drunken peck with a girl at a party last...,None,human_editor,Had a drunken peck with a girl at a party last...,None,sup4_6b_t0.7,reddit,r/relationships,Me (17F) and my boyfriend (18M) have been goin...
